In [19]:
import torch

from qtaim_embed.models.utils import load_graph_level_model_from_config
from qtaim_embed.utils.tests import get_dataset_graph_level
from qtaim_embed.data.dataloader import DataLoaderMoleculeGraphTask
from qtaim_embed.utils.data import get_default_graph_level_config
from qtaim_embed.core.dataset import HeteroGraphGraphLabelDataset

In [25]:

model_config ={
    "model": {
        "restore": True,
        "restore_path": "../../../data/saved_models/1119/qm8/model_lightning_epoch=442-val_mae=0.10.ckpt"
    }
}
model = load_graph_level_model_from_config(model_config["model"])


:::RESTORING MODEL FROM EXISTING FILE:::
... > number of tasks: 1
:::MODEL LOADED:::


In [31]:
# move model to cpu
model.cpu()

GCNGraphPred(
  (activation): ReLU()
  (embedding): UnifySize(
    (linears): ModuleDict(
      (atom): Linear(in_features=13, out_features=100, bias=False)
      (bond): Linear(in_features=9, out_features=100, bias=False)
      (global): Linear(in_features=3, out_features=100, bias=False)
    )
  )
  (conv_layers): ModuleList(
    (0-5): 6 x HeteroGraphConv(
      (mods): ModuleDict(
        (a2b): GraphConvDropoutBatch(
          (graph_conv): GraphConv(
            in=100, out=100, normalization=both
            (_activation): ReLU()
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (batch_norm): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (b2a): GraphConvDropoutBatch(
          (graph_conv): GraphConv(
            in=100, out=100, normalization=both
            (_activation): ReLU()
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (batch_norm): BatchNorm1d(100, eps=1e-05, momentum=

In [49]:
! ls ../../../data/qm8_splits_1109/test_qm8_qtaim_1109_labelled.pkl

test_qm8_qtaim_1109_labelled.pkl  train_qm8_qtaim_1109_labelled.pkl


In [54]:
qm8_loc = "../../../data/qm8_splits_1109/train_qm8_qtaim_1109_labelled.pkl"
dataset = HeteroGraphGraphLabelDataset(
    file=qm8_loc,
    allowed_ring_size=[4, 5, 6, 7],
    allowed_charges=None,
    allowed_spins=None,
    self_loop=True,
    extra_keys={
        "atom": ["extra_feat_atom_esp_total"],
        "bond": [
            "extra_feat_bond_esp_total",
            "extra_feat_bond_ellip_e_dens",
            "bond_length",
        ],
        "global": ["E1-PBE0"],
    },
    target_list=["E1-PBE0"],
    extra_dataset_info={},
    debug=False,
    log_scale_features=True,
    log_scale_targets=False,
    standard_scale_features=True,
    standard_scale_targets=True,
)

... > creating MoleculeWrapper objects


100%|██████████| 19607/19607 [00:01<00:00, 11356.04it/s]


... > bond_feats_error_count:  0
... > atom_feats_error_count:  0
element set {'O', 'C', 'F', 'H', 'N'}
selected atomic keys ['extra_feat_atom_esp_total']
selected bond keys ['extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens', 'bond_length']
selected global keys ['E1-PBE0']
... > Building graphs and featurizing


100%|██████████| 19607/19607 [00:33<00:00, 585.93it/s]


included in labels
{'global': ['E1-PBE0']}
included in graph features
{'atom': ['total_degree', 'total_H', 'is_in_ring', 'ring_size_4', 'ring_size_5', 'ring_size_6', 'ring_size_7', 'chemical_symbol_O', 'chemical_symbol_C', 'chemical_symbol_F', 'chemical_symbol_H', 'chemical_symbol_N', 'extra_feat_atom_esp_total'], 'bond': ['metal bond', 'ring inclusion', 'ring size_4', 'ring size_5', 'ring size_6', 'ring size_7', 'bond_length', 'extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens'], 'global': ['num atoms', 'num bonds', 'molecule weight']}
original loader node types: dict_keys(['atom', 'bond', 'global'])
original loader label types: dict_keys([])
include names:  dict_keys(['global'])
... > parsing labels and features in graphs


100%|██████████| 19607/19607 [00:00<00:00, 34620.10it/s]


original loader node types: dict_keys(['atom', 'bond', 'global'])
original loader label types: dict_keys(['global'])
... > Log scaling features
... > Log scaling features complete
... > Scaling features
mean [6.94480210e-01 1.98050807e-01 3.01505867e-02 1.64613505e-02
 1.04097064e-02 2.77211426e-03 5.07415527e-04 5.44933523e-02
 2.38230492e-01 9.64309161e-04 3.58360568e-01 4.10984611e-02
 8.98075974e+00]
std [0.59373438 0.36611921 0.14138506 0.10554223 0.0843036  0.04374698
 0.01874716 0.18655398 0.32920363 0.0258356  0.3463731  0.16370155
 5.65623872]
mean [0.00000000e+00 4.05843994e-02 2.21955127e-02 1.39939104e-02
 3.77720853e-03 6.79544525e-04 8.79190508e-01 6.87148685e-01
 7.57948703e-02]
std [0.         0.16273865 0.12203326 0.09748851 0.05102837 0.02169245
 0.37075581 0.20561536 0.12395626]
Standard deviation for feature 0 is 0.0, smaller than 0.001. You may want to exclude this feature.
mean [2.82349657 2.55658954 4.69597158]
std [0.17786156 0.14303875 0.08497963]
... > Scaling

In [57]:
dataset.feature_size()

{'atom': 13, 'bond': 9, 'global': 3}

In [58]:
data_loader = DataLoaderMoleculeGraphTask(
    dataset, batch_size=len(dataset.graphs), shuffle=False
)

batch_graph, batched_labels = next(iter(data_loader))
r2_pre, mae, mse = model.evaluate_manually(
    batch_graph,
    batched_labels,
    scaler_list=dataset.label_scalers,
)
print("-" * 50)
print(
    "Prior to training:\t r2: {:.4f}\t mae: {:.4f}\t mse: {:.4f}".format(
        r2_pre, mae, mse
    )
)

--------------------------------------------------
Prior to training:	 r2: 0.7773	 mae: 0.0154	 mse: 0.0226
